In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
#import torch.cuda
import random
from itertools import chain as chain
from torch.distributions.multivariate_normal import MultivariateNormal
import math

#conda activate base
cudaOn = False

In [2]:
#Data preparation

In [3]:
#tepLoc = "C:/Users/Charlie/Desktop/TEP_Data/"

#tepTrain = tepLoc + "TEP_Faulty_Training.csv"
#tepTrain = tepLoc + "TEP_FaultFree_Training.csv"

#tepTest = tepLoc + "TEP_FaultFree_Testing.csv"
#tepTest = tepLoc + "TEP_Faulty_Testing.csv"

scaler = MinMaxScaler()

#data = pd.read_csv('c172_file_1.csv')
#data = pd.read_csv(tepTrain)
#dataTest = pd.read_csv(tepTest)




data = "C:/Users/Charlie/Downloads/NGAFID_MC_C37/NGAFID_MC_C37.csv"
data = pd.read_csv(data)
data = data.fillna(0.0)

In [4]:
#lookie = dataTest[dataTest['simulationRun']==112]
#print(lookie[lookie['faultNumber']==5])
data

,volt1,volt2,amp1,amp2,FQtyL,FQtyR,E1 FFlow,E1 OilT,E1 OilP,E1 RPM,...,OAT,IAS,VSpd,NormAc,AltMSL,id,plane_id,split,date_diff,before_after
0,27.9,27.9,7.9,0.7,24.00,24.00,2.09,129.2,61.16,1191.0,...,7.2,0.0,15.74,-0.02,822.5,2,18,4,-2,1
1,27.9,27.9,7.9,0.6,24.00,24.00,2.13,129.2,61.20,1192.0,...,7.2,0.0,11.13,-0.00,822.5,2,18,4,-2,1
2,27.9,28.0,8.0,0.6,24.00,24.00,2.07,129.2,61.03,1186.0,...,7.2,0.0,-0.85,0.00,0.0,2,18,4,-2,1
3,27.9,27.9,7.8,0.6,24.00,24.00,2.12,129.2,61.16,1190.0,...,7.2,0.0,-30.64,-0.03,824.5,2,18,4,-2,1
4,27.9,27.9,7.7,0.6,24.00,24.00,2.08,129.1,61.25,1197.0,...,7.2,0.0,-23.95,-0.02,825.0,2,18,4,-2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13414562,26.6,26.7,-6.7,-0.0,14.81,19.02,1.26,152.2,52.60,764.5,...,19.5,0.0,5.91,0.06,836.0,6083,19,0,-1,1
13414563,26.5,26.6,-5.9,-0.0,14.81,19.02,1.28,152.2,52.72,771.5,...,19.5,0.0,15.54,-0.02,836.0,6083,19,0,-1,1
13414564,26.4,26.5,-5.3,-0.0,14.81,19.02,1.25,152.2,52.66,770.0,...,19.5,0.0,15.99,-0.01,836.0,6083,19,0,-1,1
13414565,26.3,26.4,-4.8,-0.0,14.86,19.05,1.26,152.2,52.66,769.0,...,19.5,0.0,7.91,0.03,836.0,6083,19,0,-1,1


In [6]:
#data = data.drop('Unnamed: 0',axis=1)
data = data.drop('date_diff',axis=1)
dataBefore = data[(data['before_after']==0)]
dataAfter = data[(data['before_after']==1)]
dataBefore = dataBefore.drop('date_diff',axis=1)
dataAfter = dataAfter.drop('date_diff',axis=1)
#data = data.drop('before_after',axis=1)

#data = data.drop('simulationRun',axis=1)
#data = data.drop('sample',axis=1)
data

,volt1,volt2,amp1,amp2,FQtyL,FQtyR,E1 FFlow,E1 OilT,E1 OilP,E1 RPM,...,E1 EGT4,OAT,IAS,VSpd,NormAc,AltMSL,id,plane_id,split,before_after
0,27.9,27.9,7.9,0.7,24.00,24.00,2.09,129.2,61.16,1191.0,...,1255.0,7.2,0.0,15.74,-0.02,822.5,2,18,4,1
1,27.9,27.9,7.9,0.6,24.00,24.00,2.13,129.2,61.20,1192.0,...,1257.0,7.2,0.0,11.13,-0.00,822.5,2,18,4,1
2,27.9,28.0,8.0,0.6,24.00,24.00,2.07,129.2,61.03,1186.0,...,1261.0,7.2,0.0,-0.85,0.00,0.0,2,18,4,1
3,27.9,27.9,7.8,0.6,24.00,24.00,2.12,129.2,61.16,1190.0,...,1267.0,7.2,0.0,-30.64,-0.03,824.5,2,18,4,1
4,27.9,27.9,7.7,0.6,24.00,24.00,2.08,129.1,61.25,1197.0,...,1271.0,7.2,0.0,-23.95,-0.02,825.0,2,18,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13414562,26.6,26.7,-6.7,-0.0,14.81,19.02,1.26,152.2,52.60,764.5,...,989.5,19.5,0.0,5.91,0.06,836.0,6083,19,0,1
13414563,26.5,26.6,-5.9,-0.0,14.81,19.02,1.28,152.2,52.72,771.5,...,988.0,19.5,0.0,15.54,-0.02,836.0,6083,19,0,1
13414564,26.4,26.5,-5.3,-0.0,14.81,19.02,1.25,152.2,52.66,770.0,...,987.0,19.5,0.0,15.99,-0.01,836.0,6083,19,0,1
13414565,26.3,26.4,-4.8,-0.0,14.86,19.05,1.26,152.2,52.66,769.0,...,986.5,19.5,0.0,7.91,0.03,836.0,6083,19,0,1


In [7]:
#dataTest = dataTest[dataTest['simulationRun']==218]
#faultNumbersT = dataTest.get('faultNumber')
#
#dataTest = dataTest.drop('Unnamed: 0',axis=1)
#dataTest = dataTest.drop('faultNumber',axis=1)
#dataTest = dataTest.drop('simulationRun',axis=1)
#dataTest = dataTest.drop('sample',axis=1)
#dataTest = data
#dataTest = dataTest.iloc(0)[0:19500] #test A and B
#dataTest = dataTest.iloc(0)[19500:38500] #test C and D
#dataTest = dataTest.iloc(0)[39000:58000]

#faultNumbersTest = []
#for i in faultNumbersT:
    #faultNumbersTest.append(i)

#data = data.astype('float64')
#dataTest = dataTest.astype('float64')

In [8]:
#run_length = 960
#num = 960
#for i in range(int(len(faultNumbersTest)/run_length)):
    #print(str(i*run_length) + ": " + str(faultNumbersTest[i*run_length]))
#print(faultNumbersTest[10000:10100])

In [9]:
print(data)
numVariables = 26#52

          volt1  volt2  amp1  amp2  FQtyL  FQtyR  E1 FFlow  E1 OilT  E1 OilP  \
0          27.9   27.9   7.9   0.7  24.00  24.00      2.09    129.2    61.16   
1          27.9   27.9   7.9   0.6  24.00  24.00      2.13    129.2    61.20   
2          27.9   28.0   8.0   0.6  24.00  24.00      2.07    129.2    61.03   
3          27.9   27.9   7.8   0.6  24.00  24.00      2.12    129.2    61.16   
4          27.9   27.9   7.7   0.6  24.00  24.00      2.08    129.1    61.25   
...         ...    ...   ...   ...    ...    ...       ...      ...      ...   
13414562   26.6   26.7  -6.7  -0.0  14.81  19.02      1.26    152.2    52.60   
13414563   26.5   26.6  -5.9  -0.0  14.81  19.02      1.28    152.2    52.72   
13414564   26.4   26.5  -5.3  -0.0  14.81  19.02      1.25    152.2    52.66   
13414565   26.3   26.4  -4.8  -0.0  14.86  19.05      1.26    152.2    52.66   
13414566   26.2   26.3  -4.8  -0.1  14.91   1.00      0.00      0.0     0.00   

          E1 RPM  ...  E1 EGT4   OAT  I

In [10]:
#Helper Methods

In [11]:
#Split and reshape the data set by step_size , use min-max or stanrdardlize method to rescale the data
def Splitting_dataset(data, step_size, scale=True, scaler_type=MinMaxScaler):
        l = len(data) 
        data = scaler_type().fit_transform(data)
        Xs = []
        Ys = []
        for i in range(0, (len(data) - step_size)):
            Xs.append(data[i:i+step_size])
            Ys.append(data[i:i+step_size])
        train_x, test_x, train_y, test_y = [np.array(x) for x in train_test_split(Xs, Ys)]
        assert train_x.shape[2] == test_x.shape[2] == (data.shape[1] if (type(data) == np.ndarray) else len(data))
        return  (train_x.shape[2], train_x, train_y, test_x, test_y)
    
def get_batch(x, batch_size):
    """Made with taking test_x or XX as input"""
    #make stochastic
    t = 0
    while t >= 0:
        x_mod = len(x) % batch_size
        start = random.random() * (len(x)-x_mod)
        start = int(start)
        if start + batch_size < len(x):
            t = t-1
    batch = torch.tensor(x[start:(start+batch_size)]) #!! added tensor line
    #print(batch.shape)
    return batch

def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

def loss_fn(recon_x, x, mu, logvar):
    #rec = torch.nan_to_num(recon_x)
    #print(recon_x)
    BCE = F.binary_cross_entropy(recon_x, x, size_average=True)
    
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu**2 -  logvar.exp())
    return BCE + KLD
    
def tep_testing_stepped(dat,step_size):
    res = []
    ind = 0
    scale = MinMaxScaler().fit(dat)
    dat = pd.DataFrame(scale.transform(dat))
    #print(int((len(data)/step_size)))
    for i in range(int((len(dat)/step_size))):
        if ind + step_size < len(dat):
            step = []
            for j in range(step_size):
              #print(data.iloc(0)[ind])#[ind])
              step.append(dat.iloc(0)[ind])
              ind = ind + 1
            res.append(step)
    return res

In [12]:
#Model

In [13]:
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=27, z_dim=31, n_flow_steps=1):
        super(VAE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(image_size, h_dim),
            nn.LeakyReLU(0.1),
            nn.Linear(h_dim, z_dim*2)
        )
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, h_dim),
            nn.ReLU(),
            nn.Linear(h_dim, image_size),
            nn.Sigmoid()
        )

    
    def reparameterize(self, mu, logvar):
        mu.double()
        logvar.double()
        std = logvar.mul(0.5).exp_() 
        esp = to_var(torch.randn(*mu.size()))
        z = mu + std * esp
        return z
    
    def forward(self, x):
        #print("------------------")
        #print(x)
        h = self.encoder(x)
        #print(h)
        #print("------------------")
        mu, logvar = torch.chunk(h, 2, dim=1)
        z = self.reparameterize(mu, logvar)
        #print(z)
        z = model(z)
        tensorZ = torch.tensor(z[0])#torch.tensor(z[0])
        #print(str(tensorZ.shape) + str(" YYY"))
        #print(z[0])
        #print("--" + str(self.decoder(tensorZ).shape))
        
        return self.decoder(tensorZ), mu, logvar
        #return self.decoder(z), mu, logvar
    

class stacked_NVP(nn.Module):
    def __init__(self, d, k, hidden, n):
        super().__init__()
        self.bijectors = nn.ModuleList([
            R_NVP(d, k, hidden=hidden) for _ in range(n)
        ])
        self.flips = [True if i%2 else False for i in range(n)]
        
    def forward(self, x):
        log_jacobs = []

        for bijector, f in zip(self.bijectors, self.flips):
            x, log_pz, lj = bijector(x, flip=f)
            log_jacobs.append(lj)
        
        return x, log_pz, sum(log_jacobs)
    
    def inverse(self, z):
        for bijector, f in zip(reversed(self.bijectors), reversed(self.flips)):
            z = bijector.inverse(z, flip=f)
        return z
    
class R_NVP(nn.Module):
    def __init__(self, d, k, hidden):
        super().__init__()
        self.d, self.k = d, k
        self.sig_net = nn.Sequential(
                    nn.Linear(k, hidden),
                    nn.LeakyReLU(),
                    nn.Linear(hidden, d - k))

        self.mu_net = nn.Sequential(
                    nn.Linear(k, hidden),
                    nn.LeakyReLU(),
                    nn.Linear(hidden, d - k))

    def forward(self, x, flip=False):
        x1, x2 = x[:, :self.k], x[:, self.k:] 
        #print("t")
        if flip:
            x2, x1 = x1, x2
        
        # forward
        sig = self.sig_net(x1)
        #print("x1: " + str(x1.shape))
        #print("x2: " + str(x2.shape))
        z1, z2 = x1, x2 * torch.exp(sig) + self.mu_net(x1)
        
        if flip:
            z2, z1 = z1, z2
        
        z_hat = torch.cat([z1, z2], dim=-1)

        log_pz = base_dist.log_prob(z_hat)
        log_jacob = sig.sum(-1)
        
        return z_hat, log_pz, log_jacob
    
    def inverse(self, Z, flip=False):
        z1, z2 = Z[:, :self.k], Z[:, self.k:] 
        
        if flip:
            z2, z1 = z1, z2
        
        x1 = z1
        x2 = (z2 - self.mu_net(z1)) * torch.exp(-self.sig_net(z1))
        
        if flip:
            x2, x1 = x1, x2
        return torch.cat([x1, x2], -1)

In [14]:
#Model Setup

In [18]:
step_size = 3
batch = 512
index_step_length = numVariables
epochs = 20

num = 5

d = numVariables
k = int(numVariables/2)

base_mu, base_cov = torch.zeros(numVariables), torch.eye(numVariables)

base_mu = torch.nn.parameter.Parameter(to_var(base_mu.double()))
base_cov = torch.nn.parameter.Parameter(to_var(base_cov.double()))
#base_mu = torch.nn.parameter.Parameter(base_mu,requires_grad=True)
#base_cov = torch.nn.parameter.Parameter(base_cov,requires_grad=True)
print(base_mu)
print(base_cov)
base_dist = MultivariateNormal(base_mu, base_cov)
#---------------------------------------------------------------------------------------------------------------------------------
labels, X, Y, XX, YY = Splitting_dataset(dataBefore, step_size)
#XX.cuda()
demo = VAE(index_step_length,h_dim=13,z_dim=numVariables)
model = stacked_NVP(d, k, hidden=512,n=num)#hidden -> 512
demo.double()
model.double()
    
#next set of tests should be with n=3, last set was with n=1
optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-3)
optimizer2 = torch.optim.Adam(demo.parameters(), lr=1e-3)
optimizer3 = torch.optim.RMSprop([base_mu,base_cov], lr=1e-3)

scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.999)
#scheduler2 = torch.optim.lr_scheduler.ExponentialLR(optimizer2, 0.999)

if torch.cuda.is_available() & cudaOn:
    demo.cuda()
    print("demo done")
    model.cuda()
    print("model done")

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,

In [19]:
#Model Training

In [20]:
idx = 0

anomaly_history = []
loss_history = []
avgSum = 0
avgCount = 0

for epoch in range(epochs):
    b = get_batch(X,batch)
    for i in range(batch):
        #print(i)
        optimizer.zero_grad()
        optimizer2.zero_grad()
        optimizer3.zero_grad()
        localX = to_var(torch.tensor(b[i]))
        recon, mu, logvar = demo(localX)
        #print(recon.shape)
        #print(localX.shape)
        loss = loss_fn(recon, localX, mu, logvar) #doing kl-divergence loss correctly
        """This bound (kl loss) provides a unified objective function for 
        op-timization of both the parameters θ and φ of the model and variational approximation, respectively."""

        loss.backward()
        
        #demo.double()
        #model.double()
        
        optimizer.step()
        optimizer2.step()
        optimizer3.step()
        scheduler.step()
        #scheduler2.step()
        idx = idx + 1

        avgSum = avgSum + torch.mean(loss/batch)
        avgCount = avgCount + 1
        anomaly_score = abs(torch.mean(localX-recon))

        if idx%step_size == 0:
            loss_history.append(avgSum/avgCount)
            anomaly_history.append(anomaly_score)
            avgSum = 0
            avgCount = 0

        if idx%100 == 0:
            print("Epoch[{}/{}] Loss: {:.3f}".format(epoch+1, epochs, loss.data.item()/batch))

C:\Users\Charlie\AppData\Local\Temp/ipykernel_31300/62241503.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  localX = to_var(torch.tensor(b[i]))


RuntimeError: size mismatch, m1: [3 x 27], m2: [26 x 13] at C:\Users\builder\AppData\Local\Temp\pip-req-build-e5c8dddg\aten\src\TH/generic/THTensorMath.cpp:136

In [ ]:
p = plt.figure()
plt.plot(loss_history,'g-',label='h 10,z 2')

In [ ]:
#stepped = tep_testing_stepped(data,3)
#test = pd.read_csv("C:/Users/Charlie/Documents/big_old_data_files/ngafid2.0-main/ngafid2.0-main/example_data/C172/log_110812_095915_KCKN.csv")
test = dataBefore[:50000]
stepped = tep_testing_stepped(test,3)

In [ ]:
fileLoc = "C:/Users/Charlie/Desktop/picsForDemo2/tepGeneratedLast/NGAFID_Test/c37_data_before.png"

#for step in stepped:
    #step = torch.tensor(step)
    #print(step.shape)


step_start = 0
anomalies = []
y_nomalies = []
vals = []
pred_vals = []
county = 0
start = 0
view = []
for step in stepped:    
    step = torch.tensor(step)
    step = step.double()
    recon,_,_ = demo(step)
    for j in range(len(step)):
        a = (recon[0][j] - step[0][j])
        anom = a**2
        #anom2 = torch.mean(torch.tensor(faultNumbersTest[(setNum*strtOfTwo)+(county*step_size):(setNum*strtOfTwo)+(county*step_size+step_size)],dtype=torch.float64))
        anomalies.append(torch.tensor(anom))
        #anomalies.append(torch.tensor(abs(anom)))
        #y_nomalies.append(anom2)
        pred_vals.append(torch.mean(recon))
        vals.append(torch.mean(step))
        step_start = step_start + 1
        county = county + 1
    for a in anomalies:
        if start+1 < len(anomalies):
            view.append(abs(anomalies[start+1].item() - a.item()))  
            start = start + 1
            
p3 = plt.figure()
plt.plot(anomalies, 'g-')
plt.savefig(fileLoc)
p4 = plt.figure()
#plt.plot(y_nomalies, 'b-')
plt.plot(vals, 'b-')
p5 = plt.figure()
plt.plot(pred_vals, 'r-')
""""""

In [ ]:
loc = "C:/Users/Charlie/Desktop/picsForDemo2/tepGeneratedLast/"

strtOfTwo = int(len(dataTest)/20)
sets = []
runNum = 0 
print(len(faultNumbersTest))

print("////////////////////")
for i in range(20):
    print(str(strtOfTwo*runNum) + " : " + str((strtOfTwo*runNum)+run_length))
    #print(faultNumbersTest[strtOfTwo*runNum])
    dt = dataTest[strtOfTwo*runNum:(strtOfTwo*runNum)+run_length]
    t1 = tep_testing_stepped(dt,step_size)
    sets.append(t1)
    runNum = runNum + 1
print("////////////////////")

setNum = 0
for set in sets:
    setName = "anom" + str(setNum+1) + "Faulty_Flow_111"
    fileLoc = loc + setName + ".png"
    step_start = 0
    anomalies = []
    y_nomalies = []
    vals = []
    pred_vals = []
    county = 0
    for step in set:
        step = to_var(torch.tensor(step,dtype=torch.float64))
        recon,_,_ = demo(step)
        for j in range(len(step)):
            a = (recon[0][j] - step[0][j])
            anom = a**2
            anom2 = torch.mean(torch.tensor(faultNumbersTest[(setNum*strtOfTwo)+(county*step_size):(setNum*strtOfTwo)+(county*step_size+step_size)],dtype=torch.float64))
            anomalies.append(torch.tensor(anom))
            #anomalies.append(torch.tensor(abs(anom)))
            y_nomalies.append(anom2)
            pred_vals.append(torch.mean(recon))
            vals.append(torch.mean(step))
            step_start = step_start + 1
            county = county + 1
    setNum = setNum + 1
    start = 0
    view = []
    max = -99999
    min = 99999
    maxA = -99999
    minA = 99999

    print(len(anomalies))

    for a in anomalies:
        if start+1 < len(anomalies):
            view.append(abs(anomalies[start+1].item() - a.item()))  
            start = start + 1

    for i in range(len(view)):
        j = i + 1
        if view[i] > max:
            max = a.item()
        if view[i] < min:
            min = a.item()
        if i < len(view)-1:
            #print("i: " + str(view[i]))
            #print("j: " + str(view[j]))
            v = abs(view[i]-view[j])
            #print("v: " + str(v))
            if v > maxA:
                maxA = v
            if v < minA:
                minA = v

    print()
    print(max)
    print(maxA)
    print(minA)

    #could get loc min and max given a step size rather than literally between individual points

    #ADD LINEAR REGRESSION LINE
    
    p3 = plt.figure()
    plt.plot(anomalies, 'g-')
    plt.savefig(fileLoc)
    p4 = plt.figure()
    #plt.plot(y_nomalies, 'b-')
    plt.plot(vals, 'b-')
    p5 = plt.figure()
    plt.plot(pred_vals, 'r-')
    print("-------------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------------")